# Imports

In [46]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

## Create Model

In [47]:
model = gp.Model("Two-Stage Stochastic Programming Model for Vessel chartering strategies for Offshore Wind Farms")

# First-Stage Model

## First-Stage Sets and Indices

In [ ]:
V = {
    0: 'Crew Transfer Vessel',
    1: 'Service Operation Vessel',
}

T = {
    0: 'January',
    # 1: 'February',
    # 2: 'March',
    # 3: 'April',
    # 4: 'May',
    # 5: 'June',
    # 6: 'July',
    # 7: 'August',
    # 8: 'September',
    # 9: 'October',
    # 10: 'November',
    # 11: 'December'
}

## First-Stage Parameters

In [49]:
C_ST = {
    0: 50,  # Cost per month for short-term chartering of Crew Transfer Vessel
    1: 150,  # Cost per month for short-term chartering of Service Operation Vessel
}

C_LT = {
    0: 3000,  # Cost for long-term chartering of Crew Transfer Vessel
    1: 28000,  # Cost for long-term chartering of Service Operation Vessel
}

## First-Stage Decision Variables

In [50]:
gamma_ST = model.addVars(V, T, vtype=GRB.INTEGER, name="gamma_ST")  # Number of short-term chartered vessels of type v in month t
gamma_LT = model.addVars(V, vtype=GRB.INTEGER, name="gamma_LT")  # Number of long-term chartered vessels of type v

# Second-Stage Model

## Second-Stage Sets and Indices

In [ ]:
#set of wind farms
W = {
    0: 'Wind Farm A',
    1: 'Wind Farm B',
}

#set of maintenance task categories
M = {
    0: 'Small Maintenance',
    1: 'Large Maintenance',
}

#set of maintenance patterns vessel v can perform
K_v = {
    0: [0],
    1: [0, 1, 2],
}

#set of days within each month t
D_t = {t: [d for d in range(t * 5, (t + 1) * 5)] for t in T}

#set of days in the planning horizon
D = [d for t in T for d in D_t[t]]
print(D)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

## Second-Stage Parameters

In [52]:
#set of scenarios
S = {
    0: 'Scenario 1',
    1: 'Scenario 2',
    2: 'Scenario 3',
}

#number of maintenance tasks of category m that a team performs in pattern k
P_mk = {
    (0, 0): 4, #(m, k)
    (0, 1): 2,
    (0, 2): 0,
    (1, 0): 0,
    (1, 1): 1,
    (1, 2): 2,
}

#time (in hours) to perform maintenance pattern k
L_k = {
    0: 4,
    1: 5,
    2: 4,
}

#time (in hours) for vessel type v to travel to wind farm w and back (round trip time)
L_vw = {
    (0, 0): 2, #(v, w)
    (0, 1): 3,
    (1, 0): 4,
    (1, 1): 5,
}

#duration (in hours) of longest weather window in scenario s on day d, 0 if too short
import random

# Terskel for minimum værvindu (f.eks. 3 timer)
min_window = 5

#duration (in hours) of longest weather window in scenario s on day d, 0 if too short
A_vwd_s = {} # (v, w, d, s) -> duration of longest weather window in hours

for v in V:
    for w in W:
        for d in D:
            for s in S:
                duration = random.randint(0, 12)  # Simulerer værvindu mellom 0 og 12 timer
                A_vwd_s[(v, w, d, s)] = duration if duration >= min_window else 0

# number of teams available of vessel type v
N_v = {
    0: 3,
    1: 5,
}

# downtime cost per day for each untreated failure
C_down = 5

# cost per day of using vessel v in wind farm w 
C_usage_vw = {
    (0, 0): 1, #(v, w)
    (0, 1): 1,
    (1, 0): 2,
    (1, 1): 2.5,
}

# initial backlog of maintenance tasks of category m at wind farm w at the start of the planning horizon
B_mw_0 = {
    (0, 0): 0, #(m, w)
    (0, 1): 0,
    (1, 0): 0,
    (1, 1): 0,
}

# number of failures of maintenance tasks of category m that occurs at wind farm w in scenario s on day d
F_mwd_s = {}

for m in M:
    for w in W:
        for d in D:
            for s in S:
                if m == 0:
                    F_mwd_s[(m, w, d, s)] = random.randint(0, 2)  # Simulerer 0-2 små vedlikeholdsoppgaver
                else:
                    F_mwd_s[(m, w, d, s)] = random.randint(0, 1)  # Simulerer 0-1 store vedlikeholdsoppgaver

## Second-Stage Decision Variables

In [53]:
#number of vessels of type v that are allocated to wind farm w in scenario s on day d
x_vwd_s = model.addVars(V, W, D, S, vtype=GRB.INTEGER, name="X_vwd_s")

#number of times a maintenance pattern k is performed by vessel type v at wind farm w on day d in scenario s
lambda_vwkd_s = {}

for v in V:
    for w in W:
        for k in K_v[v]:  # K_v er en dict: K_v[vessel_type] = [mønstre]
            for d in D:
                for s in S:
                    lambda_vwkd_s[(v, w, k, d, s)] = model.addVar(vtype=GRB.INTEGER, name=f"Y_{v}_{w}_{k}_{d}_{s}")

#number of maintenance tasks of category m that is performed at wind farm w in scenario s on day d
z_mwd_s = model.addVars(M, W, D, S, vtype=GRB.INTEGER, name="Y_mwd_s")

#number of maintenance tasks of category m that are backlogged at wind farm w in scenario s on day d
b_mwd_s = model.addVars(M, W, D, S, vtype=GRB.INTEGER, name="B_mwd_s")

## Second-Stage Objective Function

In [54]:
model.setObjective(

    #first-stage costs
    gp.quicksum(C_ST[v] * gamma_ST[v, t] for v in V for t in T) +
    gp.quicksum(C_LT[v] * gamma_LT[v] for v in V) +
    
    #expected second-stage costs
    gp.quicksum(
        gp.quicksum(C_down * b_mwd_s[m, w, d, s] for m in M for w in W for d in D) +
        gp.quicksum(C_usage_vw[v, w] * x_vwd_s[v, w, d, s] for v in V for w in W for d in D)
        for s in S
    ) / len(S),
    GRB.MINIMIZE
)

## Second-Stage Constraints

In [55]:
#consistency between allocated vessels in second stage and vessels chartered in first stage (6)
for v in V:
    for t in T:
        for d in D_t[t]:
            for s in S:
                model.addConstr(
                    gp.quicksum(x_vwd_s[v, w, d, s] for w in W) <= gamma_ST[v, t] + gamma_LT[v],
                    name=f"Vessel_Allocation_Consistency_v{v}_t{t}_d{d}_s{s}"
                )
                
#consistency between maintenance patterns performed and available teams given the number of vessels allocated (7)
for v in V:
    for w in W:
        for d in D:
            for s in S:
                model.addConstr(
                    gp.quicksum(lambda_vwkd_s[v, w, k, d, s] for k in K_v[v]) <= N_v[v] * x_vwd_s[v, w, d, s],
                    name=f"Team_Availability_Consistency_v{v}_w{w}_d{d}_s{s}"
                )

#ensure that patterns are not chosen if they cannot fit within the weather window (8)
for v in V:
    for w in W:
        for k in K_v[v]:
            for d in D:
                for s in S:
                    model.addConstr(
                        (L_k[k] + L_vw[v, w] - A_vwd_s[v, w, d, s])*lambda_vwkd_s[v, w, k, d, s] <= 0,
                        name=f"Weather_Window_Constraint_v{v}_w{w}_k{k}_"
                    )
                    
#connect maintenance tasks performed to patterns chosen (9)
for w in W:
    for m in M:
        for d in D:
            for s in S:
                model.addConstr(
                    z_mwd_s[m, w, d, s] <= gp.quicksum(P_mk[m, k] * lambda_vwkd_s[v, w, k, d, s] for v in V for k in K_v[v]),
                    name=f"Maintenance_Task_Completion_m{m}_w{w}_d{d}_s{s}"
                )

#initialize backlog at the start of the planning horizon (10)
for w in W:
    for m in M:
        for s in S:
            model.addConstr(
                b_mwd_s[m, w, 0, s] == B_mw_0[m, w],
                name=f"Initial_Backlog_m{m}_w{w}_s{s}"
            )
            
#backlog balance (11)
for w in W:
    for m in M:
        for d in D[1:]: # Start from the second day
            for s in S:
                model.addConstr(
                    b_mwd_s[m, w, d, s] == b_mwd_s[m, w, d - 1, s] + F_mwd_s[m, w, d, s] - z_mwd_s[m, w, d, s],
                    name=f"Backlog_Balance_m{m}_w{w}_d{d}_s{s}"
                )

# Solving the model

In [56]:
model.setParam("LogFile", "gurobi_log.txt")
model.optimize()

for v in model.getVars():
    if v.X > 0:  # eller bare print alle
        print(f"{v.VarName} = {v.X}")

# Print the optimal values of the decision variables
# for v in V:
#     for t in T:
#         if gamma_ST[v, t].X > 0:
#             print(f"Short-term charter {gamma_ST[v, t].X} vessels of type {V[v]} in month {T[t]}")
# for v in V:
#     if gamma_LT[v].X > 0:
#         print(f"Long-term charter {gamma_LT[v].X} vessels of type {V[v]}")
# for v in V:
#     for w in W:
#         for d in D:
#             for s in S:
#                 if x_vwd_s[v, w, d, s].X > 0:
#                     print(f"Allocate {x_vwd_s[v, w, d, s].X} vessels of type {V[v]} to wind farm {W[w]} on day {d} in scenario {S[s]}")
# for m in M:
#     for w in W:
#         for d in D:
#             for s in S:
#                 if z_mwd_s[m, w, d, s].X > 0:
#                     print(f"Perform {z_mwd_s[m, w, d, s].X} maintenance tasks of category {M[m]} at wind farm {W[w]} on day {d} in scenario {S[s]}")
# for m in M:
#     for w in W:
#         for d in D:
#             for s in S:
#                 if b_mwd_s[m, w, d, s].X > 0:
#                     print(f"Backlog {b_mwd_s[m, w, d, s].X} maintenance tasks of category {M[m]} at wind farm {W[w]} on day {d} in scenario {S[s]}")


Set parameter LogFile to value "gurobi_log.txt"
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 23760 rows, 21626 columns and 57627 nonzeros
Model fingerprint: 0xd34829a3
Variable types: 0 continuous, 21626 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [3e-01, 3e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 17977 rows and 14111 columns
Presolve time: 0.32s
Presolved: 5783 rows, 7515 columns, 18385 nonzeros
Variable types: 0 continuous, 7515 integer (53 binary)
Found heuristic solution: objective 573199.66667

Root relaxation: objective 7.130063e+03, 3539 iterations, 0.06 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wor